In [ ]:
# 2.6 Solution 2: Rééchantillonnage
print("\n--- SOLUTION 2: RÉÉCHANTILLONNAGE ---")

# Sous-échantillonnage des classes majoritaires et sur-échantillonnage des classes minoritaires
from sklearn.utils import resample

# Séparer les exemples par classe
class_samples = []
for i in range(n_classes):
    class_samples.append(imbalanced_images[imbalanced_labels == i])

# Définir la taille cible (moyenne)
target_size = int(total_samples / n_classes)
print(f"Taille cible par classe après rééchantillonnage: {target_size}")

# Rééchantillonner chaque classe
resampled_images = []
resampled_labels = []

for i in range(n_classes):
    if len(class_samples[i]) == 0:
        continue
        
    # Sur-échantillonnage pour les classes minoritaires
    if len(class_samples[i]) < target_size:
        resampled = resample(class_samples[i], 
                            replace=True,        # Avec remplacement
                            n_samples=target_size,
                            random_state=42)
    # Sous-échantillonnage pour les classes majoritaires
    elif len(class_samples[i]) > target_size:
        resampled = resample(class_samples[i],
                            replace=False,       # Sans remplacement
                            n_samples=target_size,
                            random_state=42)
    else:
        resampled = class_samples[i]
    
    resampled_images.append(resampled)
    resampled_labels.extend([i] * len(resampled))

# Convertir en arrays numpy
resampled_images = np.vstack([img for img in resampled_images if len(img) > 0])
resampled_labels = np.array(resampled_labels)

# Vérifier la nouvelle distribution
plt.figure(figsize=(10, 6))
resampled_class_counts = np.bincount(resampled_labels, minlength=n_classes)
plt.bar(range(len(class_names)), resampled_class_counts)
plt.xticks(range(len(class_names)), class_names, rotation=90)
plt.title("Distribution après rééchantillonnage")
plt.xlabel("Classe")
plt.ylabel("Nombre d'échantillons")
plt.tight_layout()
plt.show()

# Diviser en ensembles d'entraînement et de validation
X_train_res, X_val_res, y_train_res, y_val_res = train_test_split(
    resampled_images, resampled_labels, test_size=0.2, random_state=42
)

# Entraîner sur les données rééchantillonnées
resampled_model = create_base_model()
history_resampled = resampled_model.fit(
    X_train_res, y_train_res,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_res, y_val_res),
    verbose=1
)

# Évaluation du modèle rééchantillonné
resampled_predictions = resampled_model.predict(test_images)
resampled_pred_classes = np.argmax(resampled_predictions, axis=1)

# Rapport de classification
print("\nRapport de classification - Modèle avec rééchantillonnage:")
print(classification_report(test_labels, resampled_pred_classes, target_names=class_names))

# 2.7 Comparaison des solutions pour les problèmes de données
print("\n--- COMPARAISON DES SOLUTIONS POUR LES PROBLÈMES DE DONNÉES ---")

# Comparaison des performances par classe
models_data = {
    "Déséquilibré": imbalanced_pred_classes,
    "Pondération": weighted_pred_classes,
    "Rééchantillonnage": resampled_pred_classes
}

# Calculer la précision par classe pour chaque modèle
class_accuracies = {}

for model_name, predictions in models_data.items():
    accuracies = []
    for class_idx in range(n_classes):
        # Indices des exemples de cette classe
        class_indices = np.where(test_labels == class_idx)[0]
        # Précision sur cette classe
        class_acc = np.mean(predictions[class_indices] == test_labels[class_indices])
        accuracies.append(class_acc)
    class_accuracies[model_name] = accuracies

# Visualisation des précisions par classe
plt.figure(figsize=(12, 6))
x = np.arange(len(class_names))
width = 0.25
multiplier = 0

for model_name, accuracies in class_accuracies.items():
    offset = width * multiplier
    plt.bar(x + offset, accuracies, width, label=model_name)
    multiplier += 1

plt.xlabel('Classe')
plt.ylabel('Précision')
plt.title('Précision par classe et par modèle')
plt.xticks(x + width, class_names, rotation=90)
plt.legend(loc='lower center')
plt.grid(True, axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# 3. AMÉLIORATION POUR APPLICATION WEB
# ===================================

print("\n\n--- AMÉLIORATION D'UN MODÈLE POUR APPLICATION WEB ---")

# 3.1 Définition des contraintes d'une application web
print("Dans un contexte d'application web, un modèle doit être:")
print("1. Rapide (temps d'inférence court)")
print("2. Léger (taille réduite)")
print("3. Précis sur les cas d'utilisation réels")
print("4. Robuste aux variations (rotations, luminosité, etc.)")

# 3.2 Modèle de base pour une application de reconnaissance de vêtements
print("\nCréation d'un modèle de base...")

def create_web_model():
    """Crée un modèle CNN simple pour la classification de vêtements"""
    model = models.Sequential([
        # Reshaping pour CNN
        layers.Reshape((28, 28, 1), input_shape=(28, 28)),
        
        # Première couche de convolution
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Deuxième couche de convolution
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Aplatissement et couches denses
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Entraînement du modèle de base
base_web_model = create_web_model()

# Reshape pour CNN
train_images_reshaped = train_images.reshape(-1, 28, 28, 1)
test_images_reshaped = test_images.reshape(-1, 28, 28, 1)

history_base_web = base_web_model.fit(
    train_images_reshaped, train_labels,
    epochs=5,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

# 3.3 Mesure des performances initiales
print("\nÉvaluation des performances initiales...")

# Précision
test_loss, test_acc = base_web_model.evaluate(test_images_reshaped, test_labels, verbose=0)
print(f"Précision sur test: {test_acc*100:.2f}%")

# Temps d'inférence
start_time = time.time()
base_web_model.predict(test_images_reshaped[:100])
inference_time = (time.time() - start_time) / 100  # Temps moyen par image
print(f"Temps d'inférence moyen: {inference_time*1000:.2f} ms par image")

# Taille du modèle
base_web_model.save("base_web_model.h5")
import os
model_size_mb = os.path.getsize("base_web_model.h5") / (1024 * 1024)
print(f"Taille du modèle: {model_size_mb:.2f} MB")

# 3.4 Amélioration 1: Data Augmentation
print("\n--- AMÉLIORATION 1: DATA AUGMENTATION ---")

# Créer un générateur de données avec augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomFlip("horizontal"),
    layers.RandomTranslation(0.1, 0.1)
])

def create_augmented_model():
    """Crée un modèle avec data augmentation intégrée"""
    model = models.Sequential([
        # Reshaping pour CNN
        layers.Reshape((28, 28, 1), input_shape=(28, 28)),
        
        # Couche d'augmentation de données (active uniquement pendant l'entraînement)
        data_augmentation,
        
        # Première couche de convolution
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Deuxième couche de convolution
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Aplatissement et couches denses
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

augmented_model = create_augmented_model()

# Visualiser des exemples d'augmentation
plt.figure(figsize=(10, 10))
for i in range(9):
    augmented_image = data_augmentation(train_images_reshaped[i:i+1])
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_image[0, :, :, 0], cmap='gray')
    plt.title(class_names[train_labels[i]])
    plt.axis("off")
plt.tight_layout()
plt.show()

# Entraînement avec augmentation
history_augmented = augmented_model.fit(
    train_images_reshaped, train_labels,
    epochs=10,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

# 3.5 Amélioration 2: Architecture plus légère
print("\n--- AMÉLIORATION 2: ARCHITECTURE PLUS LÉGÈRE ---")

def create_lightweight_model():
    """Crée un modèle plus léger avec séparable convolutions"""
    model = models.Sequential([
        # Reshaping pour CNN
        layers.Reshape((28, 28, 1), input_shape=(28, 28)),
        
        # Première couche de convolution séparable (moins de paramètres)
        layers.SeparableConv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Deuxième couche de convolution séparable
        layers.SeparableConv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Aplatissement et couches denses
        layers.Flatten(),
        layers.Dense(64, activation='relu'),  # Plus petite que l'original
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

lightweight_model = create_lightweight_model()
lightweight_model.summary()  # Afficher le résumé pour voir la réduction de paramètres

# Entraînement du modèle léger
history_lightweight = lightweight_model.fit(
    train_images_reshaped, train_labels,
    epochs=10,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

# 3.6 Amélioration 3: Optimisation pour le déploiement
print("\n--- AMÉLIORATION 3: OPTIMISATION POUR LE DÉPLOIEMENT ---")

# Convertir en TensorFlow Lite pour le déploiement web/mobile
# (Simulé ici pour démonstration)

print("Conversion TensorFlow → TensorFlow Lite...")
converter = tf.lite.TFLiteConverter.from_keras_model(lightweight_model)
tflite_model = converter.convert()

# Écrire le modèle TFLite dans un fichier
with open('model_lite.tflite', 'wb') as f:
    f.write(tflite_model)

# Taille du modèle TFLite
tflite_model_size_mb = os.path.getsize("model_lite.tflite") / (1024 * 1024)
print(f"Taille du modèle TFLite: {tflite_model_size_mb:.2f} MB")

# Quantification (simulation)
print("\nQuantification pour réduire davantage la taille...")
converter = tf.lite.TFLiteConverter.from_keras_model(lightweight_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

# Écrire le modèle quantifié dans un fichier
with open('model_quantized.tflite', 'wb') as f:
    f.write(quantized_tflite_model)

# Taille du modèle quantifié
quantized_model_size_mb = os.path.getsize("model_quantized.tflite") / (1024 * 1024)
print(f"Taille du modèle quantifié: {quantized_model_size_mb:.2f} MB")

# 3.7 Comparaison finale des améliorations
print("\n--- COMPARAISON FINALE DES AMÉLIORATIONS ---")

# Évaluation des modèles améliorés
models_web = {
    "Base": base_web_model,
    "Augmenté": augmented_model,
    "Léger": lightweight_model
}

# Tableau comparatif
web_results = []
for name, model in models_web.items():
    # Mesurer la précision
    test_loss, test_acc = model.evaluate(test_images_reshaped, test_labels, verbose=0)
    
    # Mesurer le temps d'inférence
    start_time = time.time()
    model.predict(test_images_reshaped[:100])
    inference_time = (time.time() - start_time) / 100
    
    # Sauvegarder et mesurer la taille
    model_filename = f"{name.lower()}_model.h5"
    model.save(model_filename)
    model_size_mb = os.path.getsize(model_filename) / (1024 * 1024)
    
    web_results.append({
        "Modèle": name,
        "Précision": f"{test_acc*100:.2f}%",
        "Temps d'inférence": f"{inference_time*1000:.2f} ms",
        "Taille": f"{model_size_mb:.2f} MB"
    })

df_web_results = pd.DataFrame(web_results)
print(df_web_results)

# 3.8 Conseils pour l'intégration dans une application web
print("\n--- CONSEILS POUR L'INTÉGRATION WEB ---")
print("""
Pour intégrer efficacement votre modèle dans une application web:

1. Format de déploiement:
   - TensorFlow.js pour exécution côté client (navigateur)
   - TensorFlow Serving pour API REST côté serveur
   - TensorFlow Lite pour applications mobiles

2. Optimisations importantes:
   - Quantification pour réduire la taille et accélérer l'inférence
   - Batch processing pour les requêtes multiples
   - Mise en cache des résultats fréquents

3. Architecture recommandée:
   - API Python (Flask/FastAPI) exposant le modèle
   - Frontend JavaScript/React pour l'interface utilisateur
   - WebSockets pour les prédictions en temps réel

4. Considérations de performances:
   - Limiter la taille des images uploadées
   - Prétraitement côté client quand c'est possible
   - Monitoring des temps de réponse
""")

# 4. EXERCICE FINAL
# ================

print("\n\n--- EXERCICE FINAL: AMÉLIORATION DE MODÈLE ---")
print("""
Votre mission: Améliorer un modèle de reconnaissance de chiffres manuscrits pour une application web.

Objectifs:
1. Atteindre une précision d'au moins 98% sur MNIST
2. Réduire le temps d'inférence à moins de 10ms par image
3. Maintenir la taille du modèle sous 1 MB

Approche suggérée:
1. Commencez par analyser les performances du modèle de base
2. Identifiez les points faibles (précision, vitesse, taille)
3. Testez différentes améliorations en isolant leur impact
4. Documentez vos modifications et leurs effets
5. Préparez le modèle final pour le déploiement

Ressources:
- Dataset MNIST intégré dans TensorFlow
- Modèle de base fourni
- Documentation TensorFlow et TensorFlow Lite

Bonus:
- Créez une interface web simple pour tester le modèle
- Implémentez la reconnaissance en temps réel via la webcam
""")

# Modèle de base pour l'exercice
def create_exercise_base_model():
    """Crée un modèle de base pour l'exercice final"""
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Charger MNIST
(mnist_train_images, mnist_train_labels), (mnist_test_images, mnist_test_labels) = tf.keras.datasets.mnist.load_data()
mnist_train_images = mnist_train_images / 255.0
mnist_test_images = mnist_test_images / 255.0

# Entraîner le modèle de base
exercise_model = create_exercise_base_model()
exercise_model.fit(
    mnist_train_images, mnist_train_labels,
    epochs=3,  # Intentionnellement peu d'époques
    batch_size=128,
    validation_split=0.2,
    verbose=1
)

# Évaluer le modèle de base
test_loss, test_acc = exercise_model.evaluate(mnist_test_images, mnist_test_labels, verbose=0)
print(f"\nModèle de base pour l'exercice:")
print(f"- Précision: {test_acc*100:.2f}%")

# Temps d'inférence
start_time = time.time()
exercise_model.predict(mnist_test_images[:100])
inference_time = (time.time() - start_time) / 100
print(f"- Temps d'inférence: {inference_time*1000:.2f} ms par image")

# Taille du modèle
exercise_model.save("exercise_base.h5")
model_size_mb = os.path.getsize("exercise_base.h5") / (1024 * 1024)
print(f"- Taille du modèle: {model_size_mb:.2f} MB")

print("\nÀ vous de jouer! Améliorez ce modèle pour atteindre les objectifs.")

# 5. CONCLUSION
# ============

print("\n\n--- CONCLUSION ---")
print("""
Points clés à retenir:

1. Diagnostic:
   - Observer les courbes d'apprentissage pour détecter le surapprentissage
   - Analyser la distribution des données et les performances par classe
   - Mesurer le temps d'inférence et la taille du modèle pour applications web

2. Solutions au surapprentissage:
   - Régularisation (Dropout, L2)
   - Early Stopping
   - Augmentation de données
   - Modèles plus simples

3. Solutions aux problèmes de données:
   - Pondération des classes
   - Rééchantillonnage (sur/sous-échantillonnage)
   - Stratification des ensembles d'entraînement/validation

4. Optimisation pour le web:
   - Architectures légères (SeparableConv2D)
   - Quantification
   - TensorFlow Lite ou TensorFlow.js
   - Prétraitement efficace

L'amélioration d'un modèle est un processus itératif qui demande de:
1. Identifier précisément le problème
2. Tester une solution à la fois
3. Mesurer objectivement l'impact
4. Documenter les résultats

Ces compétences sont directement valorisables en stage et en entreprise!
""")
# Amélioration des performances de modèles de Deep Learning
# Notebook pour BTS SIO - Séance 3

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import time

print("TensorFlow version:", tf.__version__)

# 1. DIAGNOSTIC: SURAPPRENTISSAGE
# ===============================

# 1.1 Chargement du jeu de données Fashion MNIST
print("\n--- CAS 1: DIAGNOSTIC DU SURAPPRENTISSAGE ---")
print("Chargement des données...")

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

# Normalisation
train_images = train_images / 255.0
test_images = test_images / 255.0

# Noms des classes
class_names = ['T-shirt/top', 'Pantalon', 'Pull', 'Robe', 'Manteau',
               'Sandale', 'Chemise', 'Baskets', 'Sac', 'Bottine']

print(f"Dimensions des données: {train_images.shape[0]} images d'entraînement, {test_images.shape[0]} images de test")

# 1.2 Créer un modèle volontairement sujet au surapprentissage
def create_overfitting_model():
    """Crée un modèle intentionnellement sujet au surapprentissage"""
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(512, activation='relu'),  # Couche très large
        layers.Dense(512, activation='relu'),  # Seconde couche large
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# 1.3 Entraînement du modèle sujet au surapprentissage
print("Entraînement du modèle sujet au surapprentissage...")
model_overfit = create_overfitting_model()

# Utiliser peu de données pour accélérer le surapprentissage
history_overfit = model_overfit.fit(
    train_images[:6000], train_labels[:6000],
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# 1.4 Visualisation du surapprentissage
def plot_learning_curves(history, title="Courbes d'apprentissage"):
    """Trace les courbes d'apprentissage à partir de l'historique d'entraînement"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Courbe de précision
    ax1.plot(history.history['accuracy'], label='Entraînement')
    ax1.plot(history.history['val_accuracy'], label='Validation')
    ax1.set_xlabel('Époque')
    ax1.set_ylabel('Précision')
    ax1.set_title('Évolution de la précision')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Courbe de perte
    ax2.plot(history.history['loss'], label='Entraînement')
    ax2.plot(history.history['val_loss'], label='Validation')
    ax2.set_xlabel('Époque')
    ax2.set_ylabel('Perte')
    ax2.set_title('Évolution de la fonction de perte')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

plot_learning_curves(history_overfit, "Surapprentissage: Écart croissant entre entraînement et validation")

# 1.5 Évaluation sur les données de test
test_loss, test_acc = model_overfit.evaluate(test_images, test_labels, verbose=0)
print(f"Précision sur les données d'entraînement: {history_overfit.history['accuracy'][-1]*100:.2f}%")
print(f"Précision sur les données de validation: {history_overfit.history['val_accuracy'][-1]*100:.2f}%")
print(f"Précision sur les données de test: {test_acc*100:.2f}%")
print("\nNOTE: Un grand écart entre entraînement et validation/test indique un surapprentissage.")

# 1.6 Solution 1: Régularisation L2 (Weight Decay)
print("\n--- SOLUTION 1: RÉGULARISATION L2 ---")

def create_l2_model():
    """Crée un modèle avec régularisation L2"""
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(512, activation='relu', 
                    kernel_regularizer=tf.keras.regularizers.l2(0.001)),  # Ajout de L2
        layers.Dense(512, activation='relu',
                    kernel_regularizer=tf.keras.regularizers.l2(0.001)),  # Ajout de L2
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

model_l2 = create_l2_model()

history_l2 = model_l2.fit(
    train_images[:6000], train_labels[:6000],
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

plot_learning_curves(history_l2, "Avec régularisation L2: Réduction du surapprentissage")

# 1.7 Solution 2: Dropout
print("\n--- SOLUTION 2: DROPOUT ---")

def create_dropout_model():
    """Crée un modèle avec Dropout"""
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),  # Désactive aléatoirement 50% des neurones
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),  # Désactive aléatoirement 50% des neurones
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

model_dropout = create_dropout_model()

history_dropout = model_dropout.fit(
    train_images[:6000], train_labels[:6000],
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

plot_learning_curves(history_dropout, "Avec Dropout: Régularisation efficace")

# 1.8 Solution 3: Early Stopping
print("\n--- SOLUTION 3: EARLY STOPPING ---")

def create_base_model():
    """Crée un modèle de base identique au modèle de surapprentissage"""
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(512, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

model_early = create_base_model()

# Ajout du callback Early Stopping
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',    # Surveille la perte sur validation
    patience=5,            # Nombre d'époques sans amélioration avant l'arrêt
    restore_best_weights=True  # Restaure les meilleurs poids
)

history_early = model_early.fit(
    train_images[:6000], train_labels[:6000],
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

plot_learning_curves(history_early, "Avec Early Stopping: Arrêt avant la dégradation")

# 1.9 Comparaison des solutions
print("\n--- COMPARAISON DES SOLUTIONS CONTRE LE SURAPPRENTISSAGE ---")

# Évaluation sur les données de test
models = {
    "Base (surapprentissage)": model_overfit,
    "Régularisation L2": model_l2,
    "Dropout": model_dropout,
    "Early Stopping": model_early
}

# Tableau comparatif
results = []
for name, model in models.items():
    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
    results.append({
        "Modèle": name,
        "Précision Test": f"{test_acc*100:.2f}%"
    })

df_results = pd.DataFrame(results)
print(df_results)

# 2. DIAGNOSTIC: PROBLÈMES DE DONNÉES
# ==================================

print("\n\n--- CAS 2: DIAGNOSTIC DES PROBLÈMES DE DONNÉES ---")

# 2.1 Simulation de problèmes de données
print("Simulation de problèmes de données...")

# Création d'un jeu de données déséquilibré
np.random.seed(42)
n_samples = 3000

# Sélectionner principalement des T-shirts et pantalons
selected_classes = [0, 1]  # T-shirt et pantalon
mask_majority = np.isin(train_labels[:n_samples], selected_classes)
# Ajouter quelques exemples des autres classes
mask_minority = ~mask_majority
minority_indices = np.where(mask_minority)[0][:500]  # Seulement 500 exemples des autres classes

# Combiner pour créer un dataset déséquilibré
combined_indices = np.concatenate([np.where(mask_majority)[0], minority_indices])
imbalanced_images = train_images[combined_indices]
imbalanced_labels = train_labels[combined_indices]

# 2.2 Visualisation du déséquilibre
plt.figure(figsize=(10, 6))
class_counts = np.bincount(imbalanced_labels)
plt.bar(range(len(class_names)), class_counts)
plt.xticks(range(len(class_names)), class_names, rotation=90)
plt.title("Distribution déséquilibrée des classes")
plt.xlabel("Classe")
plt.ylabel("Nombre d'échantillons")
plt.tight_layout()
plt.show()

print("\nDistribution des classes:")
for i, count in enumerate(class_counts):
    print(f"{class_names[i]}: {count} échantillons ({count/len(imbalanced_labels)*100:.1f}%)")

# 2.3 Entraînement sur données déséquilibrées
print("\nEntraînement sur données déséquilibrées...")

# Diviser en ensembles d'entraînement et de validation
from sklearn.model_selection import train_test_split

X_train_imb, X_val_imb, y_train_imb, y_val_imb = train_test_split(
    imbalanced_images, imbalanced_labels, test_size=0.2, random_state=42
)

imbalanced_model = create_base_model()
history_imbalanced = imbalanced_model.fit(
    X_train_imb, y_train_imb,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_imb, y_val_imb),
    verbose=1
)

# 2.4 Évaluation du modèle déséquilibré
imbalanced_predictions = imbalanced_model.predict(test_images)
imbalanced_pred_classes = np.argmax(imbalanced_predictions, axis=1)

# Matrice de confusion
cm = confusion_matrix(test_labels, imbalanced_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title("Matrice de confusion - Modèle entraîné sur données déséquilibrées")
plt.ylabel('Valeur réelle')
plt.xlabel('Valeur prédite')
plt.tight_layout()
plt.show()

# Rapport de classification
print("\nRapport de classification - Modèle déséquilibré:")
print(classification_report(test_labels, imbalanced_pred_classes, target_names=class_names))

# 2.5 Solution: Pondération des classes
print("\n--- SOLUTION 1: PONDÉRATION DES CLASSES ---")

# Calculer les poids des classes
total_samples = len(imbalanced_labels)
n_classes = len(np.unique(imbalanced_labels))
class_weights = {}

for i in range(n_classes):
    class_weights[i] = total_samples / (n_classes * class_counts[i]) if class_counts[i] > 0 else 0

print("Poids des classes:")
for i, weight in class_weights.items():
    if class_counts[i] > 0:
        print(f"{class_names[i]}: {weight:.2f}")

# Entraîner avec des poids de classe
weighted_model = create_base_model()
history_weighted = weighted_model.fit(
    X_train_imb, y_train_imb,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_imb, y_val_imb),
    class_weight=class_weights,
    verbose=1
)

# Évaluation du modèle pondéré
weighted_predictions = weighted_model.predict(test_images)
weighted_pred_classes = np.argmax(weighted_predictions, axis=1)

# Matrice de confusion
cm_weighted = confusion_matrix(test_labels, weighted_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_weighted, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title("Matrice de confusion - Modèle avec pondération des classes")
plt.ylabel('Valeur réelle')
plt.xlabel('Valeur prédite')
plt.tight_layout()
plt.show()

# Rapport de classification
print("\nRapport de classification - Modèle avec pondération des classes:")
print(classification_report(test_labels, weighted_pred_classes, target_names=class_names))

